<a href="https://colab.research.google.com/github/sejongsmarcle/2023_Spring_Kaggle_Study/blob/main/%EB%B2%84%EC%8A%A4_%EC%9A%B4%ED%96%89%EC%8B%9C%EA%B0%84_%EC%98%88%EC%B8%A1_%ED%81%B4%EB%A1%A0%EC%BD%94%EB%94%A9_(%EB%8D%B0%EC%9D%B4%EC%BD%98_%2B_%EC%9B%90_%ED%95%AB_%EC%9D%B8%EC%BD%94%EB%94%A9)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 라이브러리 및 데이터 

- matplotlib 사용시 한국어 나오게 하기 위해서 조치를 취해줌
- 데이터 분석 및 학습에 필요한 라이브러리 가져오기
- train, test 데이터셋 import

In [1]:
## matplotlib 사용시 한국어 나오게 하는 코드

!apt -qq -y install fonts-nanum > /dev/null

# 폰트를 사용하기 위해 필요한 라이브러리
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

# 폰트 경로 설정
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

# 폰트 이름 얻어오기, 폰트 사이즈 설정
font = fm.FontProperties(fname=fontpath, size=9)

# 그래프에 retina display 적용

%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정

plt.rc('font', family='NanumBarunGothic') 



In [2]:
 import pandas as pd
 import lightgbm as lgb # 학습 알고리즘
 import folium # 지도 라이브러리

**LightGBM** : import lightgbm as lgb
- GBM : Gradient Boosting Model로 tree를 기반으로하는 학습 알고리즘
- LightGBM : 간단히 말하자면, GBM의 가벼운 버전

https://mac-user-guide.tistory.com/79 : 참조자료

In [66]:
train = pd.read_csv('/content/drive/MyDrive/229611_버스 운행시간 예측 경진대회_data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/229611_버스 운행시간 예측 경진대회_data/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/229611_버스 운행시간 예측 경진대회_data/submission_제출양식.csv')

# 2. 탐색적 자료분석
- 자료의 특성을 탐색하는게 주요함

*head, tail값을 보면서 자료의 특이점 파악*

In [4]:
train.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,06시,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,06시,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,06시,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),06시,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,07시,374.0,은남동,33.485822,126.490897,64


id : 고유 id

date : 버스 운행 날짜

route_id : 버스 노선 ID

vh_id : 버스 id

route_nm : 버스 노선 실제 번호

now_latitude : 현재 정류소의 위도

now_longitude : 현재 정류소의 경도

now_station : 현재 정류소 이름

now_arrive_time : 현재 정류장에 도착한 시간

distance : 현재 정류장에서 다음 정류장까지 실제 이동한 거리

next_station : 다음 정류소 이름

next_latitude : 다음 정류소의 위도

next_longitude : 다음 정류소의 경도

next_arrive_time : 다음 정류장에 도착할 때 까지 걸린 시간(단위:초)으로 답안 제출을 위해서 예측해야 되는 값


In [5]:
test.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude
0,210457,2019-10-29,405136001,7997025,360-1,33.457724,126.554014,제대마을,07시,333.0,제대아파트,33.458783,126.557353
1,210458,2019-10-29,405136001,7997025,360-1,33.458783,126.557353,제대아파트,07시,415.0,제주대학교,33.459893,126.561624
2,210459,2019-10-29,405136001,7997025,360-1,33.478867,126.483833,한라중학교/부영아파트,08시,417.0,대림2차아파트,33.478850,126.488350
3,210460,2019-10-29,405136001,7997025,360-1,33.478850,126.488350,대림2차아파트,08시,334.0,연동대림1차아파트,33.480700,126.489933
4,210461,2019-10-29,405136001,7997025,360-1,33.480700,126.489933,연동대림1차아파트,08시,550.0,케이티앤지,33.482077,126.485355


In [6]:
submission.head()

,id,next_arrive_time
0,210457,0
1,210458,0
2,210459,0
3,210460,0
4,210461,0


In [7]:
train.tail()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
210452,210452,2019-10-28,405328102,7983486,281-2,33.255783,126.577450,비석거리,21시,528.0,삼아아파트,33.251896,126.574417,96
210453,210453,2019-10-28,405328102,7983486,281-2,33.248595,126.568527,동문로터리,21시,280.0,매일올레시장 7번입구,33.249753,126.565959,50
210454,210454,2019-10-28,405328102,7983486,281-2,33.251891,126.560303,서귀포시 구 버스터미널,21시,114.0,아랑조을거리 입구,33.251084,126.559551,16
210455,210455,2019-10-28,405328102,7983486,281-2,33.251084,126.559551,아랑조을거리 입구,21시,223.0,평생학습관,33.249504,126.558068,38
210456,210456,2019-10-28,405328102,7983486,281-2,33.248487,126.511195,대륜동주민센터,21시,189.0,서귀포버스터미널(가상정류소),33.249091,126.509224,24


In [8]:
test.tail()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude
91769,302350,2019-11-05,405328102,7983486,281-2,33.493625,126.534764,제주지방법원(광양방면),20시,272.0,고산동산(광양방면),33.495540,126.532907
91770,302351,2019-11-05,405328102,7983486,281-2,33.495540,126.532907,고산동산(광양방면),20시,447.0,제주시청(광양방면),33.498925,126.530351
91771,302352,2019-11-05,405328102,7983486,281-2,33.498925,126.530351,제주시청(광양방면),20시,418.0,광양사거리,33.500473,126.527103
91772,302353,2019-11-05,405328102,7983486,281-2,33.500473,126.527103,광양사거리,20시,140.0,탐라장애인 종합복지관,33.500228,126.525625
91773,302354,2019-11-05,405328102,7983486,281-2,33.499916,126.521348,용천마을,20시,541.0,제주버스터미널(종점),33.499190,126.516324


In [9]:
print(train['date'].min())
print(train['date'].max())

print(test['date'].min())
print(test['date'].max())

2019-10-15
2019-10-28
2019-10-29
2019-11-05


In [58]:
# shape을 이용해서 column 개수 확인

print(train.shape)
print(test.shape)
print(submission.shape)

(210457, 14)
(91774, 13)
(91774, 2)


*info 이용해서 결측치와 데이터 형태 확인*

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210457 entries, 0 to 210456
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                210457 non-null  int64  
 1   date              210457 non-null  object 
 2   route_id          210457 non-null  int64  
 3   vh_id             210457 non-null  int64  
 4   route_nm          210457 non-null  object 
 5   now_latitude      210457 non-null  float64
 6   now_longitude     210457 non-null  float64
 7   now_station       210457 non-null  object 
 8   now_arrive_time   210457 non-null  object 
 9   distance          210457 non-null  float64
 10  next_station      210457 non-null  object 
 11  next_latitude     210457 non-null  float64
 12  next_longitude    210457 non-null  float64
 13  next_arrive_time  210457 non-null  int64  
dtypes: float64(5), int64(4), object(5)
memory usage: 22.5+ MB


In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91774 entries, 0 to 91773
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               91774 non-null  int64  
 1   date             91774 non-null  object 
 2   route_id         91774 non-null  int64  
 3   vh_id            91774 non-null  int64  
 4   route_nm         91774 non-null  object 
 5   now_latitude     91774 non-null  float64
 6   now_longitude    91774 non-null  float64
 7   now_station      91774 non-null  object 
 8   now_arrive_time  91774 non-null  object 
 9   distance         91774 non-null  float64
 10  next_station     91774 non-null  object 
 11  next_latitude    91774 non-null  float64
 12  next_longitude   91774 non-null  float64
dtypes: float64(5), int64(3), object(5)
memory usage: 9.1+ MB


*describe 이용해서 평균값, 중앙값과 같은 수치적 특징 파악*

In [13]:
train.describe()

,id,route_id,vh_id,now_latitude,now_longitude,distance,next_latitude,next_longitude,next_arrive_time
count,210457.000000,2.104570e+05,2.104570e+05,210457.000000,210457.000000,210457.000000,210457.000000,210457.000000,210457.000000
mean,105228.000000,4.052491e+08,7.988694e+06,33.434528,126.603451,490.256100,33.434711,126.603687,85.380824
std,60753.847139,9.132404e+04,6.774077e+03,0.102350,0.123961,520.563932,0.102224,0.123838,85.051170
min,0.000000,4.051360e+08,7.983000e+06,33.244382,126.473300,97.000000,33.244382,126.473300,6.000000
25%,52614.000000,4.051365e+08,7.983093e+06,33.325283,126.523900,291.000000,33.325283,126.524550,44.000000
50%,105228.000000,4.053201e+08,7.983431e+06,33.484667,126.551050,384.000000,33.484860,126.551050,66.000000
75%,157842.000000,4.053201e+08,7.997041e+06,33.500197,126.650322,542.000000,33.500228,126.650322,102.000000
max,210456.000000,4.053281e+08,7.997124e+06,33.556167,126.935188,7461.000000,33.556167,126.935188,2996.000000


**중복값을 제외한 rount_id에 대한 vh_id 개수 확인**
- 개수의 차이에 따라 배차간격 예측 가능

In [14]:
train.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,06시,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,06시,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,06시,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),06시,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,07시,374.0,은남동,33.485822,126.490897,64


In [59]:
len(train['now_station'].unique()) #unique를 이용하여 칼럼의 종류 파악

348

In [16]:
train.groupby('route_id')

In [17]:
train[['route_id','vh_id']] 

,route_id,vh_id
0,405136001,7997025
1,405136001,7997025
2,405136001,7997025
3,405136001,7997025
4,405136001,7997025
...,...,...
210452,405328102,7983486
210453,405328102,7983486
210454,405328102,7983486
210455,405328102,7983486


*route_id와 vh_id가 동일한 값들을 제거해줘야 고유값들을 정확하게 파악가능*

In [60]:
train[['route_id','vh_id']].drop_duplicates() # drop_duplicates : 중복되는 값 제외

,route_id,vh_id
0,405136001,7997025
181,405136001,7997034
381,405136001,7997035
539,405136001,7997040
717,405136001,7997044
...,...,...
20226,405136522,7997004
26037,405320116,7983225
27098,405320122,7983067
45506,405328102,7983401


In [61]:
train[['route_id','vh_id']].drop_duplicates().groupby('route_id').count() 
# 중복값을 제외한 뒤에 route_id를 기준으로 groupby 해준 뒤 개수 측정

,vh_id
route_id,
405136001,10
405136002,5
405136007,1
405136012,3
405136521,6
405136522,15
405320111,3
405320112,5
405320113,1


In [62]:
train[['route_id','next_arrive_time']].groupby('route_id').mean() 
# 위 코드에서 개수 대신 평균값 도출
# route_id 별로 next_arrive_time의 평균값이 차이난다는 점을 확인할 수 있음

,next_arrive_time
route_id,
405136001,92.063688
405136002,108.270852
405136007,94.172947
405136012,111.322633
405136521,114.673939
405136522,101.339188
405320111,68.901710
405320112,67.602392
405320113,68.403649


*버스 별로 다음 station 도착 시간이 다르다*

# 3-1. forilum 기초
- 다양한 forilum tool을 이용해 지도를 출력해본다.

In [21]:
coord = [35.1559361, 129.1384361] # 좌표 설정
map_osm = folium.Map(location = coord) # folium.map을 실행 시키기 위해서 미리 정해둔 location값 대입
map_osm #map 실행

In [22]:
map_osm = folium.Map(location=coord, zoom_start=12)
# zoom_start = 시작 시점에 줌 값을 설정 (기본값 : 10)
map_osm 

In [23]:
map_osm = folium.Map(location = coord, tiles = 'Stamen Terrain')
# tiles : map의 종류 설정하기 (stamen train : 산 지형을 위주로 보기)
map_osm

In [24]:
map_osm = folium.Map(location = coord, tiles = 'Stamen Toner')
#stamen toner : 흑백으로 보기
map_osm

In [25]:
coord = [35.2335123, 129.0810047]

map_osm = folium.Map(location=coord, zoom_start=12)

#마커 생성 (folium package 안에 'maker' 사용)
folium.Marker(coord, popup='부산대학교', tooltip = '부산대학교 tooltip').add_to(map_osm) # 마커 생성 뒤 지도에 추가해주는 기능
#popup : 클릭 했을 때 나오는 문구
#tooltip : hovering 할 때 출력할 문구
#add_to : 지도 위에 maker 올리기 위해서 사용되는 문법

#해당 지도 html form으로 출력
map_osm.save('index.html')

map_osm

In [26]:
map_osm = folium.Map(location=coord, zoom_start=12)
folium.Marker(coord, popup = '부산대학교', icon=folium.Icon(icon='star')).add_to(map_osm)
# icon : icon에 어떤 기호를 쓸껀지 결정                                                                          
map_osm

In [27]:
map_osm = folium.Map(location=[37.541, 126.986], zoom_start=11)

folium.Marker([37.566345, 126.977893], popup='서울특별시청', icon=folium.Icon(color='red',icon='info-sign')).add_to(map_osm)
# Marker 이용해서 좌표 사용해서 서울 시청에 마커 설정
folium.Circle([37.4600, 126.9519], tooltip='서울대학교', radius=3000, color="green").add_to(map_osm)
folium.CircleMarker([37.5662, 126.9386], radius=10, popup='연세대학교',color="black", fill_color="crimson").add_to(map_osm)
# circle : 단순하게 영역표시 / circlemarker : 마커 역할을 겸하는 영역표시 
# radius : circle의 반지름 설정 

map_osm

In [28]:
map_osm = folium.Map(location=[37.541, 126.986], zoom_start=11)
folium.RegularPolygonMarker([37.566345, 126.977893], popup='서울특별시청', fill_color='red',color='red', number_of_sides=3, radius=30, fill_opacity=0.2).add_to(map_osm)
folium.RegularPolygonMarker([37.4600, 126.9519], popup='서울대학교', fill_color='green',color='green', number_of_sides=4, radius=10, fill_opacity=0.2).add_to(map_osm)
folium.RegularPolygonMarker([37.5662, 126.9386], popup='연세대학교',fill_color='blue',color='blue', number_of_sides=6, radius=10).add_to(map_osm)
#RegularpolygonMarker : 구형이 아닌 마커 만들기 
#number_of_sides : 구형(=0)에서 시작하는 각의 개수
#fill_opacity : 색상 채우기 & 투명도 값 설정
map_osm

# 3-2. 지도에 버스 루트 표시하기 

In [63]:
map_data = train[['route_id','now_latitude','now_longitude','now_station']]
# 전체 train 데이터 중에 사용할 데이터 추리기 
map_bus_route = map_data[map_data['route_id'] == 405320122].drop_duplicates("now_station")
# map_data 중 route_id가 405320122인 데이터만 사용
# 그리고 nowstation이 동일한 값은 drop하기

map_osm = folium.Map(location=[33.4134, 126.5190], zoom_start = 10.5)
# 지도 기본 설정

for item in map_bus_route.index: # 버스 루트따라서 마커를 찍기위해 for문 설정
    lat = map_bus_route.loc[item,'now_latitude']
    long = map_bus_route.loc[item,'now_longitude']
    
    folium.Marker([lat,long], popup = map_bus_route.loc[item,'now_station'],
                      icon = folium.Icon(color = 'red', icon = 'info-sign')
                      ).add_to(map_osm) # 위 forlium 기초에서 배웠던 marker 파라미터 사용

map_osm.save('index.html')

map_osm

# 4. 데이터 전처리
- 결측치가 없는 대신, object 형식의 자료가 있기 때문에 이 자료들 중 주요한 object를 숫자형 자료로 바꿔주는 "원-핫 인코딩"이 주요함

In [30]:
train

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,06시,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,06시,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,06시,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),06시,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,07시,374.0,은남동,33.485822,126.490897,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210452,210452,2019-10-28,405328102,7983486,281-2,33.255783,126.577450,비석거리,21시,528.0,삼아아파트,33.251896,126.574417,96
210453,210453,2019-10-28,405328102,7983486,281-2,33.248595,126.568527,동문로터리,21시,280.0,매일올레시장 7번입구,33.249753,126.565959,50
210454,210454,2019-10-28,405328102,7983486,281-2,33.251891,126.560303,서귀포시 구 버스터미널,21시,114.0,아랑조을거리 입구,33.251084,126.559551,16
210455,210455,2019-10-28,405328102,7983486,281-2,33.251084,126.559551,아랑조을거리 입구,21시,223.0,평생학습관,33.249504,126.558068,38


*now_arrive_time은 object형이기 때문에 int형으로 바꿔줄 필요가 있음*

In [67]:
train['now_arrive_time']

0         06시
1         06시
2         06시
3         06시
4         07시
         ... 
210452    21시
210453    21시
210454    21시
210455    21시
210456    21시
Name: now_arrive_time, Length: 210457, dtype: object

In [69]:
train['now_arrive_time'].str.slice(stop = -1) 
# 뒤에서 첫번째 요소를 슬라이스해줌

0         06
1         06
2         06
3         06
4         07
          ..
210452    21
210453    21
210454    21
210455    21
210456    21
Name: now_arrive_time, Length: 210457, dtype: object

In [70]:
train['now_arrive_time'].str.slice(stop = -1).astype('int')
# 슬라이스 해준 뒤에 자료형도 int형으로 바꿔줘야함

0          6
1          6
2          6
3          6
4          7
          ..
210452    21
210453    21
210454    21
210455    21
210456    21
Name: now_arrive_time, Length: 210457, dtype: int64

In [71]:
# train과 test data set에 동일하게 저장
train['now_arrive_time'] = train['now_arrive_time'].str.slice(stop = -1).astype('int')

In [72]:
test['now_arrive_time'] = test['now_arrive_time'].str.slice(stop = -1).astype('int')

*now_station과 next_station도 object형이기 때문에 정수형으로 바꿔줄 필요가 있음*


---

**범주형 데이터 변환을 위해 LabelEncoder사용**

https://mizykk.tistory.com/10 : 자세한 내용은 이곳 참조 (첫 주 영화데이터 분석에서도 사용)


In [36]:
train.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,6,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,6,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,6,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),6,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,7,374.0,은남동,33.485822,126.490897,64


In [37]:
train.now_station

0              제주대학교입구
1                 제대마을
2                제대아파트
3            남국원(아라방면)
4                  도호동
              ...     
210452            비석거리
210453           동문로터리
210454    서귀포시 구 버스터미널
210455       아랑조을거리 입구
210456         대륜동주민센터
Name: now_station, Length: 210457, dtype: object

*value_counts를 사용하면 같은 종류의 칼럼이 몇 개 있는지 확인 가능*

In [38]:
train.now_station.value_counts()

남국원(아라방면)     3361
아라주공아파트       3098
도호동           3093
인다마을          3056
은남동           2789
              ... 
상도리 세화고등학교      25
오조리상동입구         21
세화고등학교          18
하도리 개긋물         11
전경대 입구           7
Name: now_station, Length: 348, dtype: int64

In [39]:
train.next_station.value_counts()

은남동                3519
제주여자중고등학교(아라방면)    3207
아라주공아파트            3190
인다마을               3105
제원아파트              2553
                   ... 
서귀포동부도서관             12
하도리 개긋물              10
전경대 입구                7
제주버스터미널(가상정류소)        3
제주한라대학교               2
Name: next_station, Length: 351, dtype: int64

In [40]:
test.now_station.value_counts()

아라주공아파트        1487
남국원(아라방면)      1425
인다마을           1409
도호동            1389
은남동            1367
               ... 
신효동               9
세화고등학교            5
전경대 입구            4
하도리 개긋물           2
제주한라대학교(종점)       1
Name: now_station, Length: 349, dtype: int64

In [41]:
test.next_station.value_counts()

은남동                1550
아라주공아파트            1539
인다마을               1505
제주여자중고등학교(아라방면)    1292
제원아파트              1249
                   ... 
세화고등학교                6
서귀포동부도서관              4
하도리 개긋물               2
제주한라대학교               1
제주버스터미널(가상정류소)        1
Name: next_station, Length: 350, dtype: int64

In [42]:
len(train['now_station'].unique())

348

In [43]:
len(train['next_station'].unique())

351

*unique의 개수가 더 많은 데이터셋을 기준으로 해줘야 빠지는 것 없이 전부 원핫 인코딩 될 것이라는 가정을 함.*

In [73]:
from sklearn.preprocessing import LabelEncoder
# LabelEncoding을 위한 라이브러리

le = LabelEncoder()
le = le.fit(train['next_station'])   #train['next_station']을 fit
train['now_station']=le.transform(train['now_station'])   #train['next_station']에 따라 encodin
test['now_station']=le.transform(test['now_station'])  

train['next_station']=le.transform(train['next_station'])  
test['next_station']=le.transform(test['next_station'])  

#한번에 now, next_station을 인코딩 해줘야 같은 정류장은 같은 값으로 인코딩 됨

In [74]:
train.now_station.value_counts()

41     3361
176    3098
63     3093
222    3056
221    2789
       ... 
113      25
192      21
145      18
307      11
225       7
Name: now_station, Length: 348, dtype: int64

In [75]:
train.next_station.value_counts()

221    3519
249    3207
176    3190
222    3105
229    2553
       ... 
118      12
307      10
225       7
243       3
258       2
Name: next_station, Length: 351, dtype: int64

In [76]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210457 entries, 0 to 210456
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                210457 non-null  int64  
 1   date              210457 non-null  object 
 2   route_id          210457 non-null  int64  
 3   vh_id             210457 non-null  int64  
 4   route_nm          210457 non-null  object 
 5   now_latitude      210457 non-null  float64
 6   now_longitude     210457 non-null  float64
 7   now_station       210457 non-null  int64  
 8   now_arrive_time   210457 non-null  int64  
 9   distance          210457 non-null  float64
 10  next_station      210457 non-null  int64  
 11  next_latitude     210457 non-null  float64
 12  next_longitude    210457 non-null  float64
 13  next_arrive_time  210457 non-null  int64  
dtypes: float64(5), int64(7), object(2)
memory usage: 22.5+ MB


# 변수 선택 및 모델 구축


In [48]:
features = ['now_latitude','now_station','now_arrive_time',
'distance','next_station','next_latitude','next_longitude']
target = ['next_arrive_time']

*원-핫 인코딩 한 값을 데이콘 영상에서 사용한 features에 추가함*

In [49]:
x_train, x_test, y_train = train[features], test[features], train[target]

# 모델 학습 및 검증
- 세가지 학습 모델 한번에 사용

In [77]:
# 모델 학습을 위한 라이브러리 불러오기
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from pandas.core.common import random_state

# 학습 모델 묶음과 같은 dict 만들어주기
model_dict = {
    'linear' : LinearRegression(),
    'rf' : RandomForestRegressor(random_state=0, n_jobs=1),
    'lgbm' : lgb.LGBMRegressor(random_state = 0)
}

**LGBMRegressor**

*가장 처음에서 가져온 라이브러리와 동일함*


---

Gradient Boosting 프레임워크로 Tree기반 학습 알고리즘임.

기존의 다른 Tree기반 알고리즘과 다른점은 Tree구조가 수평적으로 확장하는 다른 Tree기반 알고리즘에 비해 수직적으로 확장을 하는것에 있음.

In [78]:
model_dict.keys()

dict_keys(['linear', 'rf', 'lgbm'])

In [52]:
model_result = {}

#모델 차례대로 학습시키기
for key in model_dict.keys():
  print('#### 훈련중 ####')
  model_dict[key].fit(x_train, y_train)
  print('#### 예측중 ####')
  model_result[key] = model_dict[key].predict(x_test)

#### 훈련중 ####
#### 예측중 ####
#### 훈련중 ####


<ipython-input-52-9b8b4b9e50ac>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_dict[key].fit(x_train, y_train)


#### 예측중 ####
#### 훈련중 ####
#### 예측중 ####


In [53]:
model_result

{'linear': array([[ 81.2771562 ],
        [ 87.04543284],
        [ 96.37431657],
        ...,
        [ 94.17007489],
        [ 72.15436086],
        [103.79640256]]),
 'rf': array([ 35.36650507,  45.12237491, 135.88134374, ..., 126.37972036,
         31.84467172,  88.46184286]),
 'lgbm': array([ 41.96792148,  62.7542521 , 122.68399674, ..., 111.37487502,
         33.22893114, 104.18777992])}

*submission file copy하고 각 모델의 학습 결과에 따라 각각 저장해주기*

In [54]:
lr_submit = submission.copy()
rf_submit = submission.copy()
lgbm_submit = submission.copy()

In [55]:
lr_submit['next_arrive_time'] = model_result['linear']
rf_submit['next_arrive_time'] = model_result['rf']
lgbm_submit['next_arrive_time'] = model_result['lgbm']

In [56]:
lr_submit.to_csv('lr_submit.csv',index = False)
rf_submit.to_csv('rf_submit.csv',index = False)
lgbm_submit.to_csv('lgbm_submit.csv',index = False)